In [3]:
import numpy as np
import math
import plotly.express as px
import plotly.figure_factory as ff
import pandas as pd

In [4]:
def simulador(lambda_, mi_, simulation_total_time, deterministic=False):
    n, number_of_arrivals, waiting_line_time, system_spent_time, clients_in_system, clients_in_line, last_event_time = [0]*7
    queue, line_queue, arrivals, event = [], [], [], [0,""]
    queue.append([np.random.exponential(scale=1/lambda_), "Chegada"])
  
    exponential = (lambda mi: 1/mi) if deterministic else (lambda mi: np.random.exponential(scale=1/mi))

    while event[0] < simulation_total_time:
        last_event_time, event = event[0], queue.pop(0)
        clients_in_system += (event[0]-last_event_time) * n
        clients_in_line += (event[0]-last_event_time) * len(line_queue)

        if event[1] == "Chegada": 
            queue.append([event[0] + np.random.exponential(scale=1/lambda_), "Chegada"])
            arrivals.append(event[0])
            number_of_arrivals += 1
            n += 1

            if n == 1:
                queue.append([event[0] + exponential(mi_), "Saída"])
            if n > 1:
                line_queue.append(event[0])

        else:
            if len(line_queue) > 0:
                waiting_line_time += event[0] - line_queue.pop(0)
            n -= 1
            system_spent_time += (event[0] - arrivals.pop(0))
            
            if n > 0:
                queue.append([event[0] + exponential(mi_), "Saída"])
                
        queue.sort()
        
    avg_waiting_time = waiting_line_time / number_of_arrivals
    avg_system_time = system_spent_time / number_of_arrivals
    avg_clients_in_system = clients_in_system / event[0]
    avg_clients_in_line = clients_in_line / event[0]

    results = {
        'people_in_line': n,
        'simul_time': event[0],
        'avg_waiting_time': avg_waiting_time,
        'avg_system_time': avg_system_time,
        'number_of_arrivals': number_of_arrivals,
        'avg_clients_in_system': avg_clients_in_system,
        'avg_clients_in_line': avg_clients_in_line
    }

    return results


In [5]:
# obtendo dados simulados
data = simulador(lambda_=1, mi_= 2,simulation_total_time=100000)

In [6]:
print(f"Total de pessoas que foram ao banco na simulação: {data['number_of_arrivals']}")
print(f"Número de pessoas na fila ao final da simulação: {data['people_in_line']}")
print(f"Tempo total da simulação: {data['simul_time']:.4f}")
print(f"Tempo médio do cliente no sistema: {(data['avg_clients_in_system']):.4f}")
print(f"Tempo médio de espera na fila: {(data['avg_waiting_time']):.4f}")
print(f"Número médio de clientes no sistema: {(data['avg_clients_in_system']):.4f}")
print(f"Número médio de clientes na fila: {(data['avg_clients_in_line']):.4f}")

Total de pessoas que foram ao banco na simulação: 100058
Número de pessoas na fila ao final da simulação: 1
Tempo total da simulação: 100002.2247
Tempo médio do cliente no sistema: 0.9860
Tempo médio de espera na fila: 0.4878
Número médio de clientes no sistema: 0.9860
Número médio de clientes na fila: 0.4881


## Visualizações e Intervalo de Confiança

Vamos verificar o intervalo de confiança para o tempo médio do cliente no sistema.

In [7]:
def get_z_score(confidence_level):
    if confidence_level == 0.9:
        return 1.645
    elif confidence_level == 0.95:
        return 1.96
    elif confidence_level == 0.99:
        return

In [8]:
def simulator_confidence_interval(lambda_, mi_, sample_size, conf_level, simulation_total_time = 1000, name='avg_waiting_time', deterministic=False):
    # coletar amostra
    results = [simulador(lambda_, mi_, simulation_total_time, deterministic)[name] for _ in range(sample_size)]

    # calcular intervalo de confiança
    x_barra = np.mean(results)
    z = get_z_score(conf_level)
    s = np.std(results, ddof=1) # ddof=1 para usar o desvio padrão amostral
    limite_inferior = x_barra - z*s/math.sqrt(sample_size)
    limite_superior = x_barra + z*s/math.sqrt(sample_size)

    print(f"Intervalo de confiança a {conf_level*100:.0f}% de {name}: ({limite_inferior:.2f}, {limite_superior:.2f})")

    return (limite_inferior, limite_superior), results

In [9]:
tipo = 'avg_waiting_time'
intervalo, data = simulator_confidence_interval(lambda_= 1, mi_= 2, sample_size = 1000, conf_level = 0.95, name=tipo)

Intervalo de confiança a 95% de avg_waiting_time: (0.49, 0.50)


In [16]:
max(data)

0.904750291480372

In [10]:
fig = ff.create_distplot([data], ['Tempo no sistema'], curve_type='normal', bin_size=0.006)
fig.show()

In [11]:
import seaborn as sns

In [11]:
px.box(data, orientation='h', labels={"value": tipo})

In [12]:
# salvando dados para comparação
df = pd.DataFrame(data, columns=['simulador_2'])
df.to_csv('dados/simulador2.csv', index=False)

## Avaliando Resultados

Link para referência: https://people.revoledu.com/kardi/tutorial/Queuing/MM1-Queuing-System.html

$W$ = tempo médio que o cliente permanece no sistema (tempo de espera na fila + tempo de serviço). ${W = \frac{1}{\mu - \lambda}}$.\
$W_q$ = tempo médio que os clientes devem esperar na fila até serem atendidos. ${W_q = W - \frac{1}{\mu}}$.\
$L$ = número médio de clientes no sistema (na fila e sendo servidos). ${L = \lambda W}$\
$L_q$ = número médio de clientes na fila. ${L_q = \lambda W_q}$

### Caso ${\lambda} = 1$ e ${\mu} = 2$

*Tomando uma simulação como exemplo, temos a seguinte saída:*

Total de pessoas que foram ao banco na simulação: ``100175``\
Número de pessoas na fila ao final da simulação: ``1``\
Tempo total da simulação: ``100001.0090``\
Tempo médio do cliente no sistema: ``1.0027``\
Tempo médio de espera na fila: ``0.5001``\
Número médio de clientes no sistema: ``1.0045``\
Número médio de clientes na fila: ``0.5010``

Como vimos mais acima:
${W = \frac{1}{\mu - \lambda}}$. Logo, com ${\lambda = 1}$ e ${\mu = 2}$ temos: ${W = \frac{1}{2 - 1}}$ = 1\
${W_q = W - \frac{1}{\mu}}$. Logo, com ${W= 1}$ e ${\mu = 2}$ temos: ${W_q = 1 - \frac{1}{2}}$ = ${\frac{1}{2}}$\
${L = \lambda W}$. Logo, com ${W= 1}$ e ${\lambda = 1}$ temos: ${L = 1}$\
${L_q = \lambda W_q}$. Logo, com ${W_q= \frac{1}{2}}$ e ${\lambda = 1}$ temos: ${L_q = \frac{1}{2}}$

In [13]:
_ = simulator_confidence_interval(lambda_= 1, mi_= 2, sample_size = 1000, conf_level = 0.95, name='avg_waiting_time')

Intervalo de confiança a 95% de avg_waiting_time: (0.50, 0.51)


Os resultados condizem com os valores da simulação!!

### Caso ${\lambda} = 2$ e ${\mu} = 4$

*Tomando uma simulação como exemplo, temos a seguinte saída:*

Total de pessoas que foram ao banco na simulação: ``199062``\
Número de pessoas na fila ao final da simulação: ``0``\
Tempo total da simulação: ``100000.0340``\
Tempo médio do cliente no sistema: ``0.4973``\
Tempo médio de espera na fila: ``0.2475``\
Número médio de clientes no sistema: ``0.9898``\
Número médio de clientes na fila: ``0.4926``

Como vimos mais acima:
${W = \frac{1}{\mu - \lambda}}$. Logo, com ${\lambda = 2}$ e ${\mu = 4}$ temos: ${W = \frac{1}{4 - 2}} = \frac{1}{2}$\
${W_q = W - \frac{1}{\mu}}$. Logo, com ${W= \frac{1}{2}}$ e ${\mu = 4}$ temos: ${W_q = \frac{1}{2} - \frac{1}{4}}$ = ${\frac{1}{4}}$\
${L = \lambda W}$. Logo, com ${W= \frac{1}{2}}$ e ${\lambda = 2}$ temos: ${L = 2\frac{1}{2} = 1}$\
${L_q = \lambda W_q}$. Logo, com ${W_q= \frac{1}{4}}$ e ${\lambda = 2}$ temos: ${L_q = \frac{2}{4} = \frac{1}{2}}$

Os resultados condizem com os valores da simulação!!

In [14]:
_ = simulator_confidence_interval(lambda_= 2, mi_= 4, sample_size = 1000, conf_level = 0.95, name='avg_waiting_time')

Intervalo de confiança a 95% de avg_waiting_time: (0.25, 0.25)


### Caso ${\lambda} = 1$ e ${\mu} = 4$

*Tomando uma simulação como exemplo, temos a seguinte saída:*

Total de pessoas que foram ao banco na simulação: ``100563``\
Número de pessoas na fila ao final da simulação: ``1``\
Tempo total da simulação: ``100001.2990``\
Tempo médio do cliente no sistema: ``0.3329``\
Tempo médio de espera na fila: ``0.0832``\
Número médio de clientes no sistema: ``0.3348``\
Número médio de clientes na fila: ``0.0837``

Como vimos mais acima:
${W = \frac{1}{\mu - \lambda}}$. Logo, com ${\lambda = 1}$ e ${\mu = 4}$ temos: ${W = \frac{1}{4 - 1}} = \frac{1}{3}$\
${W_q = W - \frac{1}{\mu}}$. Logo, com ${W= \frac{1}{3}}$ e ${\mu = 4}$ temos: ${W_q = \frac{1}{3} - \frac{1}{4}}$ = ${\frac{1}{12}}$\
${L = \lambda W}$. Logo, com ${W= \frac{1}{3}}$ e ${\lambda = 1}$ temos: ${L = \frac{1}{3}}$\
${L_q = \lambda W_q}$. Logo, com ${W_q= \frac{1}{12}}$ e ${\lambda = 1}$ temos: ${L_q = \frac{1}{12}}$

Os resultados condizem com os valores da simulação!!

In [15]:
_ = simulator_confidence_interval(lambda_= 1, mi_= 4, sample_size = 1000, conf_level = 0.95, name='avg_waiting_time')

Intervalo de confiança a 95% de avg_waiting_time: (0.08, 0.08)


### Caso ${\lambda} = 1$ e ${\mu} = 2$ Determinístico

In [19]:
data = simulador(lambda_=1, mi_= 2,simulation_total_time=100000, deterministic=True)

In [20]:
print(f"Total de pessoas que foram ao banco na simulação: {data['number_of_arrivals']}")
print(f"Número de pessoas na fila ao final da simulação: {data['people_in_line']}")
print(f"Tempo total da simulação: {data['simul_time']:.4f}")
print(f"Tempo médio do cliente no sistema: {(data['avg_clients_in_system']):.4f}")
print(f"Tempo médio de espera na fila: {(data['avg_waiting_time']):.4f}")
print(f"Número médio de clientes no sistema: {(data['avg_clients_in_system']):.4f}")
print(f"Número médio de clientes na fila: {(data['avg_clients_in_line']):.4f}")

Total de pessoas que foram ao banco na simulação: 100212
Número de pessoas na fila ao final da simulação: 3
Tempo total da simulação: 100000.1335
Tempo médio do cliente no sistema: 0.7504
Tempo médio de espera na fila: 0.2488
Número médio de clientes no sistema: 0.7504
Número médio de clientes na fila: 0.2493


In [21]:
_ = simulator_confidence_interval(lambda_= 1, mi_= 2, sample_size = 1000, conf_level = 0.95, name='avg_waiting_time', deterministic=True)

Intervalo de confiança a 95% de avg_waiting_time: (0.25, 0.25)
